In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [4]:
data = pd.read_csv('data/water_quality.csv')
data.drop(columns=['Potability'], inplace=True)
inputs = data.sample(n=1, random_state=1).to_dict(orient='records')[0]
inputs

{'ph': 6.063354789891076,
 'Hardness': 160.76731128492577,
 'Solids': 14775.14559575291,
 'Chloramines': 7.484103646670236,
 'Sulfate': 305.82855327870027,
 'Conductivity': 327.27023868075764,
 'Organic_carbon': 12.309015955365226,
 'Trihalomethanes': 69.03845387066936,
 'Turbidity': 3.4673369148168574}

In [5]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [6]:
json_data = prepare_json(inputs)

endpoint = "https://water-classification.up.railway.app/v1/models/water-quality-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
prediction_value = prediction[0][0]
if prediction_value < 0.5:
    result = "Not Safe"
else:
    result = "Safe"

print(result)

Safe
